In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score 
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
%pip install transformers
from transformers import BertTokenizer, BertModel, BertForPreTraining
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

Import Data

In [ ]:
hope_list = pd.read_csv('https://raw.githubusercontent.com/jpc45/Amazon/main/hope_list.txt', header = None)

tweets= pd.read_csv('https://raw.githubusercontent.com/jpc45/Amazon/main/dataset1.csv')
tweets = tweets.drop(columns = ['Unnamed: 14'])
presence = tweets[(tweets['Presence/Absence'] == 1)]
extra_sample = presence.sample(n=28, random_state =1234)
tweets = tweets.append(extra_sample)

test_data = pd.read_csv('https://raw.githubusercontent.com/jpc45/Amazon/main/environmental%20tweets.csv')
test_data = test_data.drop(columns = ['Unnamed: 13'])
test_data = test_data.sample(n=230, random_state= 1234) #same as labeled
test_data['Fake_Label'] = 1
test_data.head()

prediction_df = pd.read_csv('https://raw.githubusercontent.com/jpc45/DS440_3a/main/environmental20tweets_BERTPredictions.csv')
prediction_df

,Unnamed: 0,Org name,Org type,Forbes rank,CSR rank,Sector,Text,tweet_type,CODED_hashtags,CODED_media,CODED_urls,favorite_count,retweet_count,Date,Fake_Label,Presence/Absence,Manual
0,25457,Viacom,profit,248,53,Media,.@VirtualTedS showcases ViacomXR and @awesomen...,original,1,1,0,19,11,7/11/19,1,0,0
1,52414,AbbVie,profit,96,61,Healthcare,We continue to invest in our pipeline to devel...,original,0,1,1,3,2,7/26/19,1,0,1
2,27378,Huntsman,profit,276,51,Chemicals,#Huntsman unveiled Araldite¬Æ 2050 and Araldit...,original,1,1,0,9,3,4/17/19,1,0,1
3,42166,Alleghany,profit,437,46,Financials,The eastbound lane of Pineview Drive in Elizab...,original,0,1,1,3,1,4/24/19,1,0,0
4,14326,Qualcomm,profit,137,57,Technology,The #Snapdragon 855-powered #GalaxyS10 5G from...,original,1,1,1,52,11,3/5/19,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,15265,Broadcom,profit,150,47,Technology,The Broadcom¬Æ BCM15000 (BCM15K) knowledge-bas...,original,0,0,1,5,1,3/27/19,1,0,0
226,11342,Northwestern Mutual,profit,111,61,Financials,"This Women‚Äôs History Month, we asked a few o...",original,0,0,1,2,0,3/29/19,1,0,1
227,28727,Lam Research,profit,287,54,Technology,Ensuring everyone feels comfortable at work is...,original,1,1,0,4,2,7/9/19,1,0,0
228,18158,American Electric Power,profit,192,55,Energy,A strong weather system blew across our Va and...,original,0,1,0,0,0,11/1/19,1,0,0


Hope Count Feature

In [ ]:
h_list = hope_list[0].tolist()
print(h_list)
text = ['tweet']
def hope_count(csv,h_list):
  csv['Hope_Count'] = 0
  for x in range(0,len(csv)):
    counter = 0
    split = csv['Text'].iloc[x].split()
    for y in split:
      for z in h_list:
        if(y == z):
          counter+=1
    csv['Hope_Count'].iloc[x] = counter

['achievement', 'ambition', 'anticipation', 'aspiration', 'belief', 'concern', 'confidence', 'desire', 'expectation', 'faith', 'goal', 'optimism', 'promise', 'prospect', 'wish', 'assumption', 'buoyancy', 'daydream', 'dependence', 'endurance', 'expectancy', 'fancy', 'fortune', 'gain', 'hopefulness', 'reliance', 'reverie', 'reward', 'rosiness', 'sanguineness', 'security', 'stock', 'utopia', 'largest', 'biggest', 'best', 'better', 'power', 'capability', 'capacity', 'function', 'influence', 'potential', 'skill', 'talent', 'advance', 'age', 'become', 'breed', 'build', 'come', 'cultivate', 'develop', 'expand', 'flourish', 'gain', 'increase', 'mature', 'multiply', 'produce', 'raise', 'rise', 'spread', 'sprout', 'swell', 'thrive', 'turn', 'widen']


Text Processing

In [ ]:
import re

# remove special characters and empty spaces in order to have a clean dataset
def processing(tweet, length):
  processed_tweets = []
  for x in range(0, length):  
      # Remove all the special characters
      processed_tweet = re.sub(r'\W', ' ', str(tweet.iloc[x]))
  
      # remove all single characters
      processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
  
      # Remove single characters from the start
      processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
  
      # Substituting multiple spaces with single space
      processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
  
      # Removing prefixed 'b'
      processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
  
      # Converting to Lowercase
      processed_tweet = processed_tweet.lower()
  
      processed_tweets.append(processed_tweet)
  return processed_tweets

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
print(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

Bert

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')
embeddings = []
test_embeddings = []
for x in range(0,230):
  inputs = tokenizer(str(tweets['Text'].iloc[x]), return_tensors="pt")
  test_inputs = tokenizer(str(test_data['Text'].iloc[x]), return_tensors="pt")
  outputs = model(**inputs)
  test_outputs = model(**test_inputs)
  embeddings.append(outputs[0][:,0,:].detach().numpy().tolist()[0])
  test_embeddings.append(test_outputs[0][:,0,:].detach().numpy().tolist()[0])

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')
pred_embeddings = []
test_embeddings = []
for x in range(0,230):
  inputs = tokenizer(str(tweets['Text'].iloc[x]), return_tensors="pt")
  test_inputs = tokenizer(str(test_data['Text'].iloc[x]), return_tensors="pt")
  outputs = model(**inputs)
  test_outputs = model(**test_inputs)
  embeddings.append(outputs[0][:,0,:].detach().numpy().tolist()[0])
  test_embeddings.append(test_outputs[0][:,0,:].detach().numpy().tolist()[0])

In [ ]:
print(accuracy_score(prediction_df['Presence/Absence'],prediction_df['Manual']))

0.6565217391304348


In [ ]:
#'''
tfidfconverter = TfidfVectorizer(max_features=1200, min_df=5, max_df= 0.7,  stop_words=stopwords)  
X1 = processing(tweets['Text'], len(tweets['Text']))
Y1 = tweets['Presence/Absence'].to_numpy()
X1 = tfidfconverter.fit_transform(X1).toarray()

X2 = processing(test_data['Text'],len(test_data['Text']))
X2 = tfidfconverter.fit_transform(X2).toarray()


'''
X1 = pd.DataFrame(embeddings).to_numpy()
Y1 = tweets['Presence/Absence'].to_numpy()
X2 = pd.DataFrame(test_embeddings).to_numpy()
'''
f1 = []
ascore = []
tp = 0
fp = 0
tn = 0
fn = 0
predictions = []
kf = KFold(n_splits=230)
kf.get_n_splits(X1)
for train_index, test_index in kf.split(X1):
  X_train, X_test = X1[train_index], X1[test_index]
  y_train, y_test = Y1[train_index], Y1[test_index]
#  X_pred, X_pred2 = X2[train_index], X2[test_index]
  classifier_RandomForestClassifier = RandomForestClassifier(n_estimators=100, random_state=0)  
  classifier_RandomForestClassifier.fit(X_train, y_train)
  pred = classifier_RandomForestClassifier.predict(X_test)
  f1.append(f1_score(y_test,pred, average = 'micro'))
  ascore.append(accuracy_score(y_test,pred))
  if(y_test[0] == 1 and pred[0] == 1):
    tp+=1
  elif(y_test[0] == 0 and pred[0] == 1):
    fp+=1
  elif(y_test[0] == 0 and pred[0] == 0):
    tn+=1
  else:
    fn+=1 
#  predictions.append(classifier_RandomForestClassifier.predict(X2))
print('F1 average', sum(f1)/230)
print('Accuracy Score', sum(ascore)/230)
print("True Positive: ", tp)
print("False Positive: ", fp)
print("True Negative: ", tn)
print("False Negative: ", fn)

F1 average 0.8478260869565217
Accuracy Score 0.8478260869565217
True Positive:  93
False Positive:  13
True Negative:  102
False Negative:  22


In [ ]:
X1 = pd.DataFrame(embeddings).to_numpy()
Y1 = tweets['Presence/Absence'].to_numpy()
X2 = pd.DataFrame(test_embeddings).to_numpy()

f1 = []
ascore = []
tp = 0
fp = 0
tn = 0
fn = 0
counter = 0
predictions = []
kf = KFold(n_splits=230)
kf.get_n_splits(X1)
for train_index, test_index in kf.split(X1):
  X_train, X_test = X1[train_index], X1[test_index]
  y_train, y_test = Y1[train_index], Y1[test_index]
#  X_pred, X_pred2 = X2[train_index], X2[test_index]
  clf = AdaBoostClassifier(n_estimators=100, random_state=0)
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  f1.append(f1_score(y_test,pred, average = 'micro'))
  ascore.append(accuracy_score(y_test,pred))
  if(y_test[0] == 1 and pred[0] == 1):
    tp+=1
  elif(y_test[0] == 0 and pred[0] == 1):
    fp+=1
  elif(y_test[0] == 0 and pred[0] == 0):
    tn+=1
  else:
    fn+=1 
  counter+=1
  if(counter%10 == 0):
    print(counter)
print('F1 average', sum(f1)/230)
print('Accuracy Score', sum(ascore)/230)
print("True Positive: ", tp)
print("False Positive: ", fp)
print("True Negative: ", tn)
print("False Negative: ", fn)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
F1 average 0.808695652173913
Accuracy Score 0.808695652173913
True Positive:  94
False Positive:  23
True Negative:  92
False Negative:  21


In [ ]:
X1 = pd.DataFrame(embeddings).to_numpy()
Y1 = tweets['Presence/Absence'].to_numpy()
X2 = pd.DataFrame(test_embeddings).to_numpy()

f1 = []
ascore = []
tp = 0
fp = 0
tn = 0
fn = 0
counter = 0
predictions = []
kf = KFold(n_splits=230)
kf.get_n_splits(X1)
for train_index, test_index in kf.split(X1):
  X_train, X_test = X1[train_index], X1[test_index]
  y_train, y_test = Y1[train_index], Y1[test_index]
#  X_pred, X_pred2 = X2[train_index], X2[test_index]
  clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  f1.append(f1_score(y_test,pred, average = 'micro'))
  ascore.append(accuracy_score(y_test,pred))
  if(y_test[0] == 1 and pred[0] == 1):
    tp+=1
  elif(y_test[0] == 0 and pred[0] == 1):
    fp+=1
  elif(y_test[0] == 0 and pred[0] == 0):
    tn+=1
  else:
    fn+=1 
  counter+=1
  if(counter%10 == 0):
    print(counter)
print('F1 average', sum(f1)/230)
print('Accuracy Score', sum(ascore)/230)
print("True Positive: ", tp)
print("False Positive: ", fp)
print("True Negative: ", tn)
print("False Negative: ", fn)

In [ ]:
results_float = sum(predictions)/230
results_binary = list(map(int, results_float))
print(sum(results_binary))

11


In [ ]:
test_data['Presence/Absence'] = results_binary
print(sum(test_data['Presence/Absence']))

11


In [ ]:
#test_data.to_csv('/content/drive/My Drive/Colab Notebooks/environmental20tweets_BERTPredictions.csv', index=False)
from google.colab import files
test_data.to_csv('environmental20tweets_BERTPredictions.csv') 
files.download('environmental20tweets_BERTPredictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
hope_count(tweets,h_list)
hope_count(test_data,h_list)

X3 = tweets[['Hope_Count','CODED_hashtags']]
Y3 = tweets['Presence/Absence']
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, Y3, test_size=0.2, random_state=0)
X4 = test_data[['Hope_Count','CODED_hashtags']]
Y4 = test_data['Fake_Label']


classifier_RandomForestClassifier = RandomForestClassifier(n_estimators=100, random_state=0)  
classifier_RandomForestClassifier.fit(X_train3, y_train3)
pred2 = classifier_RandomForestClassifier.predict(X_test3)
print(roc_auc_score(y_test3,pred2))
print(accuracy_score(y_test3,pred2))

test_predictions = classifier_RandomForestClassifier.predict(X4)
print(test_predictions)
print(sum(tweets['Presence/Absence']))
print(sum(test_predictions))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0.6430020283975659
0.5652173913043478
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
115
24


In [ ]:
tfidfconverter = TfidfVectorizer(max_features=1200, min_df=5, max_df= 0.7,  stop_words=stopwords)  

X1 = processing(tweets['Text'], len(tweets['Text']))
Y1 = tweets['Presence/Absence']
dataset1 = tfidfconverter.fit_transform(X1).toarray()
X_train1, X_test1, y_train1, y_test1 = train_test_split(dataset1, Y1, test_size=0.2, random_state=0)

X2 = processing(test_data['Text'],len(test_data['Text']))
Y2 = test_data['Fake_Label']
dataset2 = tfidfconverter.fit_transform(X2).toarray()
X_train2, X_test2, y_train2, y_test2 = train_test_split(dataset2, Y2, test_size=0.2, random_state=0)

In [ ]:
classifier_RandomForestClassifier = RandomForestClassifier(n_estimators=100, random_state=0)  
classifier_RandomForestClassifier.fit(X_train1, y_train1)
predictions_RandomForestClassifier = classifier_RandomForestClassifier.predict(X_test1)
print(predictions_RandomForestClassifier)
print(f1_score(y_test1,predictions_RandomForestClassifier, average = 'micro'))
print(confusion_matrix(y_test1,predictions_RandomForestClassifier))
print(accuracy_score(y_test1, predictions_RandomForestClassifier))

[0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1
 1 1 0 1 1 0 0 1 1]
0.8913043478260869
[[16  1]
 [ 4 25]]
0.8913043478260869


In [ ]:
X_test2.shape

(46, 107)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(random_state=0)
Classification = clf.fit(X_train1, y_train1)
pred = clf.predict(X_test1)
print(pred)
#cross_val_score(clf, iris.data, iris.target, cv=10)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]
